In [72]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, TFAutoModel

In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [97]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
example_tweet = "The NEET exams show our Govt in a poor light: unresponsiveness to genuine concerns; admit cards not delivered to aspirants in time; failure to provide centres in towns they reside, thus requiring unnecessary & risky travels. What a disgrace to treat our #Covid warriors like this!"

In [6]:
input = tokenizer.encode(example_tweet, return_tensors="tf")

In [89]:
output = model.predict(input)[0]

1/1 [==============================] - 2s 2s/step


In [93]:
out = model.predict(input)

1/1 [==============================] - 2s 2s/step


In [94]:
out

TFSequenceClassifierOutput(loss=None, logits=array([[-0.22857673,  0.25285968, -0.19593516]], dtype=float32), hidden_states=None, attentions=None)

In [8]:
prediction = tf.nn.softmax(output, axis=1).numpy()

In [9]:
sentiment = np.argmax(prediction)

In [10]:
print(prediction)
print(sentiment)

[[0.5976062  0.40239385]]
0


In [11]:
model.fit

<bound method Model.fit of <transformers.models.bert.modeling_tf_bert.TFBertForSequenceClassification object at 0x7f8638d05be0>>

In [12]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [14]:
data = pd.read_csv("../../data/labelled_text.csv", encoding='ISO-8859-1', header=None,)

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [17]:
texts, labels = list(data[data[2] != "Irrelevant"][3]), list(data[data[2] != "Irrelevant"][2])

In [18]:
len(texts), len(labels)

(61692, 61692)

In [19]:
labels_numeric = []
for label in labels:
    if label == "Neutral":
        labels_numeric.append(0)
    elif label == "Positive":
        labels_numeric.append(1)
    else:
        labels_numeric.append(2)

In [20]:
len(texts), len(labels_numeric)

(61692, 61692)

In [32]:
#Clean NaNs
clean_texts, clean_labels = [], []
for i, text in enumerate(texts):
    if isinstance(text, str):
        clean_texts.append(text)
        clean_labels.append(labels_numeric[i])

In [57]:
text_data = []

for text in clean_texts:
    text_data.append(tokenizer.encode(text, return_tensors="tf")[0])

In [47]:
len(clean_texts), len(clean_labels)

(61121, 61121)

In [98]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=tf.keras.metrics.Accuracy()
)
    

In [65]:
text_data[0]

<tf.Tensor: shape=(20,), dtype=int32, numpy=
array([  101, 10047,  2893,  2006,  3675,  8653,  1998,  1045,  2097,
        4028,  2017,  2035,  1010,   102,     0,     0,     0,     0,
           0,     0], dtype=int32)>

In [59]:
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    text_data, padding="post", maxlen=20
)

In [60]:
labels_data = tf.convert_to_tensor(clean_labels)
text_data = tf.convert_to_tensor(padded_inputs)

In [66]:
labels_data.shape, text_data.shape

(TensorShape([61121]), TensorShape([61121, 20]))

In [99]:
history = model.fit(x=text_data, y=labels_data, batch_size=16, epochs=1, verbose=2)

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7f8653255940>
Traceback (most recent call last):
  File "/Users/q541063/opt/anaconda3/envs/task/lib/python3.8/weakref.py", line 345, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 
Exception ignored in: <function ScopedTFGraph.__del__ at 0x7f8651879700>
Traceback (most recent call last):
  File "/Users/q541063/opt/anaconda3/envs/task/lib/python3.8/site-packages/tensorflow/python/framework/c_api_util.py", line 55, in __del__
    self.deleter(self.graph)
KeyboardInterrupt: 


KeyboardInterrupt: 